# Importing Libraries

In [1]:
!pip install keras_applications
!pip install git+https://github.com/dineshssdn-867/keras-squeezenet

  Cloning https://github.com/dineshssdn-867/keras-squeezenet to c:\users\admin\appdata\local\temp\pip-req-build-t5j_ppvm
  Created wheel for keras-squeezenet: filename=keras_squeezenet-0.4-py3-none-any.whl size=4445 sha256=e7940e8ac9d2ac18a1c059123e4f693fbcc06d8f920c03339b3ba76ec53400dc
  Stored in directory: C:\Users\Admin\AppData\Local\Temp\pip-ephem-wheel-cache-cqgw10f6\wheels\13\f4\01\39782362fc66d3a951582336d26d86f9e80a35557b7f337780
Successfully built keras-squeezenet


  Running command git clone -q https://github.com/dineshssdn-867/keras-squeezenet 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-t5j_ppvm'


In [2]:
import cv2
import tensorflow 
import tensorflow as tf
from keras_squeezenet import SqueezeNet
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout
from tensorflow.keras.layers import Flatten, BatchNormalization, Conv2D,AvgPool2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, ZeroPadding2D, Convolution2D
from tensorflow.keras.layers import ReLU, DepthwiseConv2D, Reshape, Permute, MaxPool2D, concatenate, Add
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics.pairwise import cosine_similarity

# Setting up MobileNetV2 Model

In [11]:
trained_model=MobileNetV2(input_shape=(160,160,3),           
                    include_top=False,
                    weights=None)   


trained_model.trainable=True  #Un-Freeze all the pretrained layers of 'MobileNetV2 for Training.
    
last_layer=trained_model.get_layer('out_relu')   
last_layer_output=last_layer.output 
x = tf.keras.layers.GlobalAveragePooling2D()(last_layer_output)   # Add Global Average Pooling Layer
x = tf.keras.layers.Dropout(0.8)(x)      # Add a Dropout layer
x = Dense(120)(x)
output = Dense(105,activation="softmax")(x)
model_mobilenet = Model(trained_model.input,output)

In [12]:
model_mobilenet.load_weights('model_mobilenet.h5')      # Loading the saved weights

In [13]:
model_mobilenet.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 80, 80, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 80, 80, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 80, 80, 32)   0           ['bn_Conv1[0][0]']         

 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 20, 20, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_depthwise_relu (ReLU)  (None, 20, 20, 144)  0           ['block_3_depthwise_BN[0][0]']   
                                                                                                  
 block_3_project (Conv2D)       (None, 20, 20, 32)   4608        ['block_3_depthwise_relu[0][0]'] 
                                                                                                  
 block_3_project_BN (BatchNorma  (None, 20, 20, 32)  128         ['block_3_project[0][0]']        
 lization)

 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 10, 10, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 10, 10, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_7_depthwise_BN (BatchNor  (None, 10, 10, 384)  1536       ['block_7_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_7_d

 block_10_project_BN (BatchNorm  (None, 10, 10, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 10, 10, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_expand_BN (BatchNorma  (None, 10, 10, 576)  2304       ['block_11_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_11_expand_relu (ReLU)    (None, 10, 10, 576)  0           ['block_11_expand_BN[0][0]']     
                                                                                                  
 block_11_

                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 5, 5, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 5, 5, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_project_BN (BatchNorm  (None, 5, 5, 160)   640         ['block_14_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_14_add (Add)             (None, 5, 5, 160)    0           ['block_13_project_BN[0][0]',    
                                                                  'block_14_project_BN[0][0]']    
          

In [14]:
model_mobilenet_facenet = Model(model_mobilenet.input,model_mobilenet.get_layer('dense_2').output)   # Removing the last classification layer

In [15]:
model_mobilenet_facenet.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 80, 80, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 80, 80, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 80, 80, 32)   0           ['bn_Conv1[0][0]']         

 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 20, 20, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_depthwise_relu (ReLU)  (None, 20, 20, 144)  0           ['block_3_depthwise_BN[0][0]']   
                                                                                                  
 block_3_project (Conv2D)       (None, 20, 20, 32)   4608        ['block_3_depthwise_relu[0][0]'] 
                                                                                                  
 block_3_project_BN (BatchNorma  (None, 20, 20, 32)  128         ['block_3_project[0][0]']        
 lization)

 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 10, 10, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 10, 10, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_7_depthwise_BN (BatchNor  (None, 10, 10, 384)  1536       ['block_7_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_7_d

 block_10_project_BN (BatchNorm  (None, 10, 10, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 10, 10, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_expand_BN (BatchNorma  (None, 10, 10, 576)  2304       ['block_11_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_11_expand_relu (ReLU)    (None, 10, 10, 576)  0           ['block_11_expand_BN[0][0]']     
                                                                                                  
 block_11_

                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 5, 5, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 5, 5, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_project_BN (BatchNorm  (None, 5, 5, 160)   640         ['block_14_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_14_add (Add)             (None, 5, 5, 160)    0           ['block_13_project_BN[0][0]',    
                                                                  'block_14_project_BN[0][0]']    
          

# Setting up SqueezeNet Model

In [3]:
squeezeNet = SqueezeNet()

for layer in squeezeNet.layers:
    layer.trainable = True

x = Dense(120)(squeezeNet.layers[-2].output)
output = Dense(105,activation="softmax")(x)
model_squeezeNet = Model(squeezeNet.input,output)

In [4]:
model_squeezeNet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 227, 227, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 113, 113, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 relu_conv1 (Activation)        (None, 113, 113, 64  0           ['conv1[0][0]']                  
                                )                                                             

 on)                                                                                              
                                                                                                  
 fire5/concat (Concatenate)     (None, 27, 27, 256)  0           ['fire5/relu_expand1x1[0][0]',   
                                                                  'fire5/relu_expand3x3[0][0]']   
                                                                                                  
 pool5 (MaxPooling2D)           (None, 13, 13, 256)  0           ['fire5/concat[0][0]']           
                                                                                                  
 fire6/squeeze1x1 (Conv2D)      (None, 13, 13, 48)   12336       ['pool5[0][0]']                  
                                                                                                  
 fire6/relu_squeeze1x1 (Activat  (None, 13, 13, 48)  0           ['fire6/squeeze1x1[0][0]']       
 ion)     

                                                                                                  
 relu_conv10 (Activation)       (None, 13, 13, 1000  0           ['conv10[0][0]']                 
                                )                                                                 
                                                                                                  
 global_average_pooling2d (Glob  (None, 1000)        0           ['relu_conv10[0][0]']            
 alAveragePooling2D)                                                                              
                                                                                                  
 dense (Dense)                  (None, 120)          120120      ['global_average_pooling2d[0][0]'
                                                                 ]                                
                                                                                                  
 dense_1 (

In [5]:
model_squeezeNet.load_weights('model_SqueezeNet.h5')    # Loading the saved weights

In [7]:
model_squeezeNet_facenet = Model(model_squeezeNet.input,model_squeezeNet.get_layer('dense').output)   # Removing the last classification layer

In [8]:
model_squeezeNet_facenet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 227, 227, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1 (Conv2D)                 (None, 113, 113, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 relu_conv1 (Activation)        (None, 113, 113, 64  0           ['conv1[0][0]']                  
                                )                                                           

 on)                                                                                              
                                                                                                  
 fire5/concat (Concatenate)     (None, 27, 27, 256)  0           ['fire5/relu_expand1x1[0][0]',   
                                                                  'fire5/relu_expand3x3[0][0]']   
                                                                                                  
 pool5 (MaxPooling2D)           (None, 13, 13, 256)  0           ['fire5/concat[0][0]']           
                                                                                                  
 fire6/squeeze1x1 (Conv2D)      (None, 13, 13, 48)   12336       ['pool5[0][0]']                  
                                                                                                  
 fire6/relu_squeeze1x1 (Activat  (None, 13, 13, 48)  0           ['fire6/squeeze1x1[0][0]']       
 ion)     

                                                                                                  
 relu_conv10 (Activation)       (None, 13, 13, 1000  0           ['conv10[0][0]']                 
                                )                                                                 
                                                                                                  
 global_average_pooling2d (Glob  (None, 1000)        0           ['relu_conv10[0][0]']            
 alAveragePooling2D)                                                                              
                                                                                                  
 dense (Dense)                  (None, 120)          120120      ['global_average_pooling2d[0][0]'
                                                                 ]                                
                                                                                                  
Total para

# Function to Calculate Embeddings of images from the database

In [9]:
import os
def calculate_embeddings(path, model, input_shape):
    embeddings = []      # It will be list of lists; will store list of type [embedding vector, label of image]
    for folder in os.listdir(path):
        text = folder
        folder_path = path+text+"/"
        for img in os.listdir(folder_path):
            img_path = folder_path + img
            img = cv2.imread(img_path)
            img = cv2.resize(img, input_shape,interpolation = cv2.INTER_LINEAR)
            img = tf.expand_dims(img, axis=0)
            img_predict = model.predict(img)
            emb_item = [img_predict, text]
            embeddings.append(emb_item)
    return embeddings

# Demonstration of Real Time Face Recognition using Mobilnet

In [ ]:
import cv2
import tensorflow as tf

# Calculate embeddings of dataset
embeddings = calculate_embeddings("images/", model_mobilenet_facenet, (160, 160))

# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Capture video from webcam. 
cap = cv2.VideoCapture(0)

text = ''
while True:
    # Read the frame
    _, img = cap.read()
    # Detect the faces
    faces = face_cascade.detectMultiScale(img, 1.1, 4)
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        cam_img = img[y1:y2,x1:x2]
        cam_img = cv2.resize(cam_img, (160, 160),interpolation = cv2.INTER_LINEAR)
        cam_img = tf.expand_dims(cam_img, axis=0)
        img_predict_2 = model_mobilenet_facenet.predict(cam_img)    # Create embeddings of the detected face
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        maxVal = 0.0
        # Check for similarity with embeddings of the images in the database
        for embedding in embeddings:
            val = cosine_similarity(embedding[0], img_predict_2)
            if val > maxVal:
                maxVal = val
                text = embedding[1]
        text = text + " (" + str(maxVal) + ")"    
        # Check if maximum similarity crosses the threshold or not
        if (maxVal < 0.8):
            text = "Not Recognized"
        cv2.putText(img, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    # Display
    cv2.imshow('img', img)
    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
        
# Release the VideoCapture object
cap.release()
cv2.destroyAllWindows()

# Demonstration of Real Time Face Recognition using SqueezeNet 

In [10]:
import cv2
import tensorflow as tf

# Calculate embeddings of dataset
embeddings = calculate_embeddings("images/", model_squeezeNet_facenet, (227, 227))

# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# To capture video from webcam. 
cap = cv2.VideoCapture(0)

text = ''
while True:
    # Read the frame
    _, img = cap.read()
    # Detect the faces
    faces = face_cascade.detectMultiScale(img, 1.1, 4)
    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        x1 = x
        y1 = y
        x2 = x+w
        y2 = y+h
        img1 = img[y1:y2,x1:x2]
        img1 = cv2.resize(img1, (227, 227),interpolation = cv2.INTER_LINEAR)
        img2 = tf.expand_dims(img1, axis=0)
        img_predict_2 = model_squeezeNet_facenet.predict(img2)   # Create embeddings of the detected face
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
        maxVal = 0.0
        # Check for similarity with embeddings of the images in the database
        for embedding in embeddings:
            val = cosine_similarity(embedding[0], img_predict_2)
            if val > maxVal:
                maxVal = val
                text = embedding[1] 
        # Check if maximum similarity crosses the threshold or not
        if (maxVal < 0.8):
            text = "Not Recognized"
        text = text + " (" + str(maxVal) + ")"
        cv2.putText(img, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    # Display
    cv2.imshow('img', img)
    # Stop if escape key is pressed
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
# Release the VideoCapture object
cap.release()
cv2.destroyAllWindows()